In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w', encoding="utf-8") as fo:
    fo.write(article)

Use BeautifulSoup to find all the tables

In [3]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada:_M.html', encoding="utf8").read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

Search through the tables for the one with the headings we want

In [4]:
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

Build the dataframe row by row, then manipulate the data into the shape we want

In [5]:
import pandas as pd

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=['Postcode', 'Borough', 'Neighbourhood'])

# Drop the rows with Borough 'Not assigned'
df_filt = df[df['Borough'] != 'Not assigned']

# Concatenate the neighbourhoods of boroughs with multiple entries
df_filt_g = df_filt.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Replace 'Not assigned' neighbourhoods with the Borough
df_filt_g['Neighbourhood'].replace('Not assigned', df_filt_g['Borough']).reset_index()
df_filt_g

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df_filt_g.shape

(103, 3)

In [7]:
# Read in CSV of locations of postal codes

locations = pd.read_csv('https://cocl.us/Geospatial_data')

# Rename 'Postal Code' to 'Postcode'
locations.columns = ['Postcode', 'Latitude', 'Longitude']

In [8]:
locations.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_filt_g_locs = pd.merge(df_filt_g, locations, on = 'Postcode')
df_filt_g_locs.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Create a map of Toronto with neighbourhoods superimposed on top

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_filt_g_locs['Latitude'], df_filt_g_locs['Longitude'], df_filt_g_locs['Borough'], df_filt_g_locs['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [42]:
toronto_data = df_filt_g_locs
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [43]:
toronto_data.shape

(103, 5)

In [44]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'AMW1XUWHOJDF5ADOFUJOY0OWQNUNQSYU4FLSISCEJSZJQ0KC' # your Foursquare ID
CLIENT_SECRET = 'ANCHWAVARYCRW2JKQQ4FKIT4ODETVLKDPMHHJMOS3XAPMDTP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: AMW1XUWHOJDF5ADOFUJOY0OWQNUNQSYU4FLSISCEJSZJQ0KC
CLIENT_SECRET:ANCHWAVARYCRW2JKQQ4FKIT4ODETVLKDPMHHJMOS3XAPMDTP


In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
toronto_venues = getNearbyVenues(names=toronto_data.Neighbourhood,
                                   latitudes=toronto_data.Latitude,
                                   longitudes=toronto_data.Longitude
                                  )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:47<00:00,  2.59it/s]


In [48]:
print(toronto_venues.shape)
toronto_venues.head()

(2268, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


I can check the number of venues per neighbourhood

In [49]:
toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide, King, Richmond                     100
First Canadian Place, Underground city       100
Ryerson, Garden District                     100
St. James Town                               100
Chinatown, Grange Park, Kensington Market    100
Name: Venue, dtype: int64

In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Let us now take the venue category information and create a dataframe with a one hot encoding of these data.

In [51]:
Toronto_OHE = pd.get_dummies(toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = toronto_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
Toronto_OHE.shape

(2268, 276)

I can compute the average number of venue categories per neighborhood

In [53]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue
0,"Adelaide, King, Richmond",0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


For neighborhood clustering let us select only the N most frequent venue categories per neighborhood

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Gym,Restaurant,Burger Joint,Asian Restaurant
1,Agincourt,Sandwich Place,Lounge,Chinese Restaurant,Skating Rink,Breakfast Spot,Airport Terminal,Airport Food Court,Airport Gate,Airport Lounge,Airport Service
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Music Venue,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Coffee Shop,Pharmacy,Beer Store,Sandwich Place,Discount Store,Fried Chicken Joint
4,"Alderwood, Long Branch",Pizza Place,Pool,Coffee Shop,Gym,Pharmacy,Pub,Sandwich Place,Skating Rink,Arts & Crafts Store,Art Museum


# Neighborhood clustering

Before clustering, let us use a PCA decomposition to reduce the noise of the signal and improve clustering efficiency

In [56]:
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

The dimensionality reduction has kept only a reduced number of the total dimensions

In [57]:
Toronto_grouped_clustering.shape

(100, 275)

With reduced dimensional dataset we carry out the KMeans clustering

In [58]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[1 1 0 1 1 1 1 1 1 1]
(100,)


Let us now create a dataframe that containes the neighborhood, the location and the cluster information, together with the top 10 venues

In [59]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = toronto_data.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,"Rouge, Malvern",0.0,0.0,0.0,0.0,...,Print Shop,Fast Food Restaurant,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,...,Construction & Landscaping,Bar,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,...,Intersection,Mexican Restaurant,Pizza Place,Medical Center,Electronics Store,Rental Car Location,Breakfast Spot,Airport,Airport Food Court,Airport Gate
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Woburn,0.0,0.0,0.0,0.0,...,Coffee Shop,Korean Restaurant,Music Venue,Airport Terminal,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Cedarbrae,0.0,0.0,0.0,0.0,...,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Bakery,Bank,Caribbean Restaurant,Thai Restaurant,Art Gallery,Art Museum,Aquarium


In [60]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analyze clusters

## Cluster 1

Cluster 1 contains many restaurants and diners, as well as some other services, like bus and train stations

In [61]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Playground,Park,Music Venue,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
23,Park,Bank,Convenience Store,Music Venue,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
25,Fast Food Restaurant,Food & Drink Shop,Park,Music Venue,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal
30,Airport,Park,Music Venue,Accessories Store,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
31,Grocery Store,Shopping Mall,Bank,Park,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service


## Cluster 2

Cluster 2 seems to be dominated by art galleries, music venues and airport facilities

In [62]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Print Shop,Fast Food Restaurant,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop
1,Construction & Landscaping,Bar,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop
2,Intersection,Mexican Restaurant,Pizza Place,Medical Center,Electronics Store,Rental Car Location,Breakfast Spot,Airport,Airport Food Court,Airport Gate
3,Coffee Shop,Korean Restaurant,Music Venue,Airport Terminal,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant
4,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Bakery,Bank,Caribbean Restaurant,Thai Restaurant,Art Gallery,Art Museum,Aquarium


## Cluster 3

Cluster 3 is mostly linked to the airport

In [64]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Rental Car Location,Drugstore,Aquarium,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Music Venue


## Cluster 4

Cluster 4 is the most homogeneous one (it only contains three neighborhoods) and seems to be dominated by the Baseball field

In [65]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Food Truck,Baseball Field,American Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Music Venue
91,Baseball Field,Business Service,Deli / Bodega,Pool,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal
97,Baseball Field,Music Venue,American Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop


## Cluster 5

Cluster 5 is characterized by shops and restaurants as well as the airport

In [66]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Playground,Convenience Store,Music Venue,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
48,Playground,Music Venue,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
